In [141]:
import pymongo
import numpy as np
import pandas as pd
import datetime,time
import ast

In [142]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']

Connected successfully!!!


In [143]:
def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """
    
    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']
    return df

In [144]:
df_users = read_mongo(db, "users_noRT")

In [145]:
df_users.head() ##nomes veig que users_noRT contigui columna "name"

,name
0,lluisegura
1,illoquepereza
2,ReyTravis199
3,Willopoter
4,capitalradiocv


In [146]:
df_tweets = read_mongo(db, "tweets_noRT")

In [147]:
df_tweets.head()

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,source,text,truncated,user
0,None,None,Fri Nov 09 13:30:18 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,1,False,None,1060887306708680709,1060887306708680709,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com/download/iphone"" r...",@pepcalabona Rendeix-te. No tens escapatòria,False,"{'id': 92003453, 'id_str': '92003453', 'name':..."
1,None,None,Fri Nov 09 17:03:37 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1060940987890044928,1060940987890044928,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com/download/android"" ...",@JornadaPucela A quien alineo? Plano o Carrillo,False,"{'id': 933378351267504128, 'id_str': '93337835..."
2,None,None,Fri Nov 09 07:25:49 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1060795581831020544,1060795581831020544,...,False,{'created_at': 'Wed Nov 07 13:50:29 +0000 2018...,1.060168e+18,1060167608597708806,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Ja deia Aristòtil que la primera condició per ...,True,"{'id': 92003453, 'id_str': '92003453', 'name':..."
3,None,None,Thu Nov 08 14:16:00 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,4,False,None,1060536416042659846,1060536416042659846,...,NaN,NaN,NaN,NaN,2,False,"<a href=""http://twitter.com/download/iphone"" r...","Hola @emparmoliner , de veritat fas servir ""ag...",True,"{'id': 92003453, 'id_str': '92003453', 'name':..."
4,None,None,Tue Nov 06 21:57:06 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1059927681050034178,1059927681050034178,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com/download/iphone"" r...",@iaia_toneta Mentre no venguin el patata Suare...,False,"{'id': 92003453, 'id_str': '92003453', 'name':..."


In [148]:
def tweets_FC_RT(df):
    import re
    import pandas as pd
    l = list(df["text"])
    filtr = ["RT" not in i for i in l]
    df = df[filtr].reset_index(drop=True)
    
    #Tweet =[re.sub("[(\n)(\r)]"," ",tweet) for tweet in df["text"]]
    #entities = [tweet for tweet in df["entities"]]
    FC = [tweet for tweet in df["favorite_count"]]
    RT = [tweet for tweet in df["retweet_count"]]
    #id_ = [tweet for tweet in df["id"]]
    j_user = [tweet for tweet in df["user"]]
    
    
    #CA = df["created_at"]
    #CA_l = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(CA[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(CA))]
    
    #output = pd.DataFrame([id_,Tweet,j_user,CA_l,entities,FC,RT]).T
    output = pd.DataFrame([j_user,FC,RT]).T
    output.reset_index(drop=True)
    #output.columns = ["id", "Tweet", "j_user","created_at", "entities", "FC","RT"]
    output.columns = ["j_user","FC","RT"]
    return output

In [149]:
output = tweets_FC_RT(df_tweets)

In [150]:
output.head()

,j_user,FC,RT
0,"{'id': 92003453, 'id_str': '92003453', 'name':...",1,0
1,"{'id': 933378351267504128, 'id_str': '93337835...",0,0
2,"{'id': 92003453, 'id_str': '92003453', 'name':...",0,0
3,"{'id': 92003453, 'id_str': '92003453', 'name':...",4,2
4,"{'id': 92003453, 'id_str': '92003453', 'name':...",0,0


In [151]:
#users is a dictionary that has assigned to each key (user name), a vector of three values: total FC, total RT, comptador
users = {}
numl = len(output)
names = [] #names is a list that contains the different names that 
for line in range(numl):
    nom = output['j_user'][line]['name']
    if nom in users.keys():
        users[nom][0] += output['FC'][line]
        users[nom][1] += output['RT'][line]
        users[nom][2] += 1
    else:
        users[nom] = [0,0,0]
        users[nom][0] = output['FC'][line]
        users[nom][1] = output['RT'][line]
        users[nom][2] = 1
        names.append(nom)

In [152]:
users

{'Lluís Segura': [38, 17, 11],
 'Futbolisto inexperto': [2, 0, 11],
 'p  a  b  l   o': [477, 191, 11],
 'ManuelSuarez43': [0, 0, 10],
 'Willopoter': [48, 138, 11],
 'capitalradiocv': [11, 9, 10],
 'Beñat Zarrabeitia': [104, 30, 11],
 'Yzzen': [14, 3, 11],
 'Las Rozas CF': [242, 57, 11],
 'Maria Helena Bonilla Patiño': [8, 2, 11],
 'Pablo Izquierdo': [29, 1, 11],
 'José Ramon Badoch': [17, 0, 11],
 'patriciocaamañocasal': [10, 1, 11],
 'Pablo': [29, 4, 15],
 'Nido3z /Jose': [0, 0, 8],
 'Francisco Gago Nieto': [0, 0, 11],
 'Sara (antes @saritadm)': [9, 3, 11],
 'Àncora Cultura del Mar': [17, 11, 11],
 'Deliness': [2, 0, 11],
 'frente atlético': [17, 5, 11],
 'Bea': [2, 0, 11],
 'Pedro Alsina Mier': [39, 22, 11],
 'iismailife': [13, 12, 11],
 'Lola Indigo | divademimi': [96, 23, 11],
 'carlos_marketing': [16, 5, 11],
 'Luis Armando Villanueva Rocha': [2, 0, 11],
 'Isabel_LAM': [2, 0, 11],
 'Jackoman': [11, 0, 11],
 'maud': [13, 1, 11],
 'Diego.': [8, 1, 10],
 'Diversport': [13, 11, 11],
 

In [167]:
means = pd.DataFrame(columns=['name','mean of FC','mean of RT'])
compt = 0
for name in names:
    mean_FC = users[name][0]/users[name][2]
    mean_RT = users[name][1]/users[name][2]
    #noudf = pd.DataFrame([[name,mean_FC,mean_RT]],columns=['name','mean of FC','mean of RT'])
    means.loc[compt]=[name,mean_FC,mean_RT]
    compt +=1

In [168]:
means.head()

,name,mean of FC,mean of RT
0,Lluís Segura,3.454545,1.545455
1,Futbolisto inexperto,0.181818,0.000000
2,p a b l o,43.363636,17.363636
3,ManuelSuarez43,0.000000,0.000000
4,Willopoter,4.363636,12.545455


In [170]:
high_FC = means[means['mean of FC'] > 1000]
high_FC.head()

,name,mean of FC,mean of RT


In [171]:
high_RT = means[means['mean of RT'] > 1000]
high_RT.head()

,name,mean of FC,mean of RT
